In [26]:
%%writefile arquivo.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <mpi.h>
#include <sys/time.h>

//#define start 1
//#define end 15000

int gcd(int u, int v) {
	if (v == 0)
		return u;
	return gcd(v, u % v);
}

int main(int argc, char **argv) {
  
  int rank, size; 
  long int n_elem_proc;
  int i;
  long int last;
  long int *num,*den,*elem;
	struct timeval inic,fim;

  MPI_Init(&argc,&argv);
  MPI_Comm_size(MPI_COMM_WORLD,&size);
  MPI_Comm_rank(MPI_COMM_WORLD,&rank);

  int start, end;
  start=atoi(argv[1]);
  end=atoi(argv[2]);
  if (start <= 0 || end <= start){
    if(rank==0){  
      printf("Start deve ser maior que 0 e End deve ser maior que Start");
    }
    MPI_Finalize();
    return 0;
  }


  gettimeofday(&inic,0);
  if (rank == 0){ //mestre
		printf("Numeros de %d ate %d\n", start, end);

    last = end - start + 1; // calcula o indice do ultimo elemento

    num = (long int*) calloc(sizeof(long int), last);
    den = (long int*) calloc(sizeof(long int), last);
    elem = (long int*) calloc(sizeof(long int), last);

    n_elem_proc = last/size; // numero de elementos por processo

    for(int i=start,j=0; i <= end; i++,j++){
        elem[j]= i; // adiciona o valor do elemento a ser calculado ao vetor de elementos
    } 
	}
  
  MPI_Bcast(&n_elem_proc, 1, MPI_LONG, 0, MPI_COMM_WORLD); // transmite o n de elementos para os processos slaves
  MPI_Bcast(&last, 1, MPI_LONG, 0, MPI_COMM_WORLD); // transmite o indice do ultimo elemento para os slaves

  long int k, *recbuf, *t_num,*t_den;
  int *sendcounts, *displs;
  
  recbuf = (long int*) calloc(sizeof(long int), (n_elem_proc));
  t_num = (long int*) malloc(sizeof(long int) * (n_elem_proc));
  t_den = (long int*) malloc(sizeof(long int) * (n_elem_proc));
  sendcounts = (int*) malloc(sizeof(int) * (size));
  displs = (int*) malloc(sizeof(int) * (size));
  
  k=0;
  for (i=0; i<size; i++)
    {
      if (i<size-1) sendcounts[i] = n_elem_proc;
      else sendcounts[i] = last-k;
      displs[i] = k;
      k = k+sendcounts[i];
    }
  if(rank==0){
      printf("send count \n");
      for (i=0; i<size; i++){
          printf("%d\n", sendcounts[i]);
      }
      printf("displs  \n");
      for (i=0; i<size; i++){
          printf("%d\n", displs[i]);
      }
  }

  //ScatterV para tratar quando o número de elementos não for divisível pelo número de processos
  //Sendcounts é o número de elementos para cada processo, acessado por sendcounts[rank]
  //Displs é a primeira posição do vetor que o processo receberá
  MPI_Scatterv(elem, sendcounts, displs, MPI_LONG, recbuf, sendcounts[rank], MPI_LONG, 0, MPI_COMM_WORLD); 
  
	long int factor;
	long int sum, done, n;

  printf("send count %d no rank %d\n", sendcounts[rank], rank);
	for (i = 0; i < sendcounts[rank]; i++) {
		sum = 1 + recbuf[i]; // adiciona 1 e o proprio elemento a soma do numerador 
		done = recbuf[i]; 
		factor = 2;
		while (factor < (done/2)+1) {  // faz a soma dos elementos divisiveis por i
			if ((recbuf[i] % factor) == 0) {
				sum += (factor);
			}
			factor++;
		}
		t_num[i] = sum; // recebe a soma do elemento
		t_den[i] = recbuf[i]; // proprio numero
		n = gcd(t_num[i], t_den[i]); // mdc
		t_num[i] /= n; // simplifica a fraçao
		t_den[i] /= n; // simplifica a fraçao
	} 

  MPI_Gatherv(t_num, sendcounts[rank], MPI_LONG, num, sendcounts, displs, MPI_LONG, 0, MPI_COMM_WORLD); // Envia de volta os calculos de numerador para o rank root
  MPI_Gatherv(t_den, sendcounts[rank], MPI_LONG, den, sendcounts, displs, MPI_LONG, 0, MPI_COMM_WORLD); // Envia de volta os calculos de denominador para o rank root
  
  int count = 0; 
  if(rank == 0) {
    for (i = 0; i < last; i++) {
      for (int j = i + 1; j < last; j++) {
        if ((num[i] == num[j]) && (den[i] == den[j])){
            count+=1;
            printf("%ld e %ld sao amigaveis\n", elem[i], elem[j]); // printa numeros mutuamente amigaveis
        }
      }
	  }
	}
  printf("%d",count);
  gettimeofday(&fim,0);
  if(rank==0){
    printf("\nElapsed time:%f sec\n", (fim.tv_sec+fim.tv_usec/1000000.) - (inic.tv_sec+inic.tv_usec/1000000.));
    
    free(num);
    free(den);
    free(elem);
  }
  MPI_Finalize();

  free(recbuf);
  free(t_num);
  free(t_den);
  free(sendcounts);
  free(displs);

	return EXIT_SUCCESS;
}

Overwriting arquivo.c


In [27]:
!mpicc arquivo.c -o m1 && mpiexec --mca orte_base_help_aggregate 0 --allow-run-as-root -n 4 m1 1 15001

Numeros de 1 ate 15001
send count 
3750
3750
3750
3751
displs  
0
3750
7500
11250
send count 3750 no rank 1
send count 3750 no rank 2
send count 3750 no rank 0
send count 3751 no rank 3
1 e 6 sao amigaveis
1 e 28 sao amigaveis
1 e 496 sao amigaveis
1 e 8128 sao amigaveis
6 e 28 sao amigaveis
6 e 496 sao amigaveis
6 e 8128 sao amigaveis
12 e 234 sao amigaveis
28 e 496 sao amigaveis
28 e 8128 sao amigaveis
30 e 140 sao amigaveis
30 e 2480 sao amigaveis
30 e 6200 sao amigaveis
40 e 224 sao amigaveis
42 e 3472 sao amigaveis
56 e 3724 sao amigaveis
60 e 1170 sao amigaveis
66 e 308 sao amigaveis
66 e 5456 sao amigaveis
78 e 364 sao amigaveis
78 e 6448 sao amigaveis
80 e 200 sao amigaveis
84 e 270 sao amigaveis
84 e 1488 sao amigaveis
84 e 1638 sao amigaveis
102 e 476 sao amigaveis
102 e 8432 sao amigaveis
114 e 532 sao amigaveis
114 e 9424 sao amigaveis
120 e 672 sao amigaveis
132 e 2574 sao amigaveis
135 e 819 sao amigaveis
138 e 644 sao amigaveis
138 e 11408 sao amigaveis
140 e 2480 sao am